In [13]:
# https://github.com/lingjzhu/clap-ipa/tree/main
import librosa
import numpy as np
clap_ipa_sr = 16000
some_audio,sampling_rate = librosa.load("tutorials/audio-samples/angelo-reloading.mp3")
some_audio = librosa.resample(some_audio, orig_sr=sampling_rate, target_sr=16000)
sampling_rate = 16000

In [9]:
#empty array 
audio_array = np.empty([])

word_detected = False


actions = []
scores = [0.998,0.01,0.22]
for x in macro_functions.keys():
    actions.append(x)

#actions = ["nothing","clap","snap",]


#CLAP model's prediction function
def classify(audio):
    global audio_array, word_detected, actions
    i = 0
    
    samplerate, array = audio
    
    #converts the input live audio array to numpy array and appends it to current audio array
    live_array = np.array(array)
    audio_array = np.append(audio_array, live_array)
    
    dimension = audio_array.shape
    
    
    #removes 5 second of audio from array if it's longer than 5 seconds
    if((dimension[0]) > (240000)):
        audio_array = audio_array[-240000:]
        print("Audio size: ", audio_array.shape)
    
    #Pass audio to model for predictions
    result = pipe(audio_array, candidate_labels=actions)
    
    #Formats the result to a dataframe and retrives the highest scoring label
    df = pd.DataFrame(result)
    max = df.idxmax(numeric_only=True)
    index = int(max[0])
    word = df.iat[index,1]
    print(df)
    print(word)
    
    #Checks if the highest scored label was an action label
    if word != "nothing":
        word_detected = True
        print("word was detected: ", word)
        print("pressing macros...")
        macro_functions[word]()
        
   
    #if word is detected or if the audio reaches 5 seconds, 
    #then reset the audio array to empty
    if (word_detected == True):
        audio_array = np.empty([1,1])
        print("All contents of audio array was removed.")
        word_detected = False
    
    return df

In [11]:
import sys 
sys.path.append("./clap-ipa")
from clap.encoders import *
import torch.nn.functional as F
from transformers import DebertaV2Tokenizer, AutoProcessor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

speech_encoder = SpeechEncoder.from_pretrained('anyspeech/clap-ipa-tiny-speech')
phone_encoder = PhoneEncoder.from_pretrained('anyspeech/clap-ipa-tiny-phone')
phone_encoder.eval().to(device)
speech_encoder.eval().to(device)

tokenizer = DebertaV2Tokenizer.from_pretrained('charsiu/IPATokenizer')
processor = AutoProcessor.from_pretrained('openai/whisper-tiny')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0.07146668434143066
tensor([0.0451])
0.0760946273803711
tensor([0.1298])
0.07309556007385254
tensor([0.5351])


In [12]:
import time

def ipa_converted_vals(labels, audio):
    ipa_val_list = []

    for x in range(len(labels)):
        start = time.time()
        audio_input = processor(audio,return_attention_mask=True,sampling_rate=16000,max_length=32000,return_tensors='pt')
        ipa_input = tokenizer(ipa.convert(labels[x], keep_punct=True), return_token_type_ids=False, return_tensors='pt')
        with torch.no_grad():
            speech_embed = speech_encoder(**audio_input).pooler_output
            phone_embed = phone_encoder(**ipa_input).pooler_output

        similarity = F.cosine_similarity(speech_embed,phone_embed,dim=-1)
        ipa_val_list.append(similarity)

    return ipa_val_list